#Visualization of the intensity curve of bikes from 10 eco-meters placed in Montpellier 

    Our objective for this part is to visualize the intensity of the bicycles passing a contage point in 10 deferens position in Montpellier.

    In our dataset, the bicycles are counted every day. we will consider the date of the end of days (which would be the start of the next day)

    For each count point, we will plot the intensity variation curve as a function of time (days)

    To achieve our goal, we start with several transformations and manipulations on the raw databases, and finish by programming a widgets in which we can choose the desired position (between the 10 count points). 

In [88]:
import sys
print(sys.version)

3.8.8 (default, Feb 24 2021, 15:54:32) [MSC v.1928 64 bit (AMD64)]


In [89]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from matplotlib import pyplot as plt
import json
from pandas import json_normalize
from ipywidgets import interact
from download import download



In [51]:
#Database import 
url = [
#Celleneuve
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20042633_archive.json',
#Lattes 2
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20042634_archive.json',
#Berracasa
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H19070220_archive.json',
#Lavérune
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20042632_archive.json',
#Lattes 1
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20042635_archive.json',
#Vieille poste
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20063161_archive.json',
#Gerhardt
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20063162_archive.json',
#Tanneurs
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_XTH19101158_archive.json',
#Delmas 1
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20063163_archive.json',
#Delmas 2
    'https://data.montpellier3m.fr/sites/default/files/ressources/MMM_EcoCompt_X2H20063164_archive.json',
]



download(url[0], "./data_visulization/Celleneuve.json", replace=False)
download(url[1], "./data_visulization/Lattes2.json", replace=False)
download(url[2], "./data_visulization/Berracasa.json", replace=False)
download(url[3], "./data_visulization/Lavérune.json", replace=False)
download(url[4], "./data_visulization/Lattes1.json", replace=False)
download(url[5], "./data_visulization/Vieille_poste.json", replace=False)
download(url[6], "./data_visulization/Gerhardt.json", replace=False)
download(url[7], "./data_visulization/Tanneurs.json", replace=False)
download(url[8], "./data_visulization/Delmas1.json", replace=False)
download(url[9], "./data_visulization/Delmas2.json", replace=False)



Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.

file_sizes: 100%|███████████████████████████| 29.8k/29.8k [00:00<00:00, 544kB/s]
Successfully downloaded file to ./data_visulization/Berracasa.json

file_sizes: 100%|███████████████████████████| 28.4k/28.4k [00:00<00:00, 430kB/s]
Successfully downloaded file to ./data_visulization/Lavérune.json

file_sizes: 100%|███████████████████████████| 28.6k/28.6k [00:00<00:00, 591kB/s]
Successfully downloaded file to ./data_visulization/Lattes1.json

file_sizes: 100%|███████████████████████████| 29.0k/29.0k [00:00<00:00, 712kB/s]
Successfully downloaded file to ./data_visulization/Vieille_poste.json

file_sizes: 100%|███████████████████████████| 29.0k/29.0k [00:00<00:00, 509kB/s]
Successfully downloaded file to ./data_visulization/Gerhardt.json

file_sizes: 100%|███████████████████████████| 30.6k/30.6k [00:00<0

'./data_visulization/Delmas2.json'

In [95]:
#Read .Json data as dataframe
bike_traffic_df1 = pd.read_json('./data_visulization/Celleneuve.json', lines=True)
bike_traffic_df2 = pd.read_json('./data_visulization/lattes2.json', lines=True)
bike_traffic_df3 = pd.read_json('./data_visulization/Berracasa.json', lines=True)
bike_traffic_df4 = pd.read_json('./data_visulization/Lavérune.json', lines=True)
bike_traffic_df5 = pd.read_json('./data_visulization/Lattes1.json', lines=True)
bike_traffic_df6 = pd.read_json('./data_visulization/Vieille_poste.json', lines=True)
bike_traffic_df7 = pd.read_json('./data_visulization/Gerhardt.json', lines=True)
bike_traffic_df8 = pd.read_json('./data_visulization/Tanneurs.json', lines=True)
bike_traffic_df9 = pd.read_json('./data_visulization/Delmas1.json', lines=True)
bike_traffic_df10 = pd.read_json('./data_visulization/Delmas2.json', lines=True)



In [96]:
#-------- 
from pandas import Series


#Split 'dateObserved' column into two columns 
data_test=bike_traffic_df1.join(bike_traffic_df1['dateObserved'].apply(lambda x: Series(x.split('/'))))
data_test=data_test.rename(columns = {0: 'start_of_day', 1: 'end_of_day'}) 

#replace the 'T' between date and time with a space 
data_test['end_of_day'] = data_test['end_of_day'].str.replace('T',' ')

#Convert to time series 
time_improved = pd.to_datetime(data_test['end_of_day'] , format='%Y-%m-%d %H:%M:%S')               

#Indexing data by 'end_of-day' (time series)
data_test['end_of_day'] = time_improved
data_test = data_test.set_index(['end_of_day'])

In [97]:
# Rename intensity column by position name
bike_traffic_df1=bike_traffic_df1.rename(columns = {'intensity': 'Celleneuve'})
bike_traffic_df2=bike_traffic_df2.rename(columns = {'intensity': 'Lattes2'}) 
bike_traffic_df3=bike_traffic_df3.rename(columns = {'intensity': 'Berracasa'}) 
bike_traffic_df4=bike_traffic_df4.rename(columns = {'intensity': 'Lavérune'}) 
bike_traffic_df5=bike_traffic_df5.rename(columns = {'intensity': 'Lattes1'}) 
bike_traffic_df6=bike_traffic_df6.rename(columns = {'intensity': 'Vieille_poste'}) 
bike_traffic_df7=bike_traffic_df7.rename(columns = {'intensity': 'Gerhardt'}) 
bike_traffic_df8=bike_traffic_df8.rename(columns = {'intensity': 'Tanneurs'}) 
bike_traffic_df9=bike_traffic_df9.rename(columns = {'intensity': 'Delmas1'}) 
bike_traffic_df10=bike_traffic_df10.rename(columns = {'intensity': 'Delmas2'}) 

#Extract the renamed columns, and combine them into a new dataframe 
data_test22 = pd.DataFrame((bike_traffic_df1['Celleneuve'], bike_traffic_df2['Lattes2'],bike_traffic_df3['Berracasa'],bike_traffic_df4['Lavérune'],bike_traffic_df5['Lattes1'],bike_traffic_df6['Vieille_poste']))
data_test22=data_test22.T

#Indexing neww data by 'end_of-day' (time series)
data_test22['startday'] = time_improved
data_test22 = data_test22.set_index(['startday'])
print(data_test22)




            Celleneuve  Lattes2  Berracasa  Lavérune  Lattes1  Vieille_poste
startday                                                                    
2020-12-18        53.0    405.0      524.0      69.0    158.0          127.0
2020-12-19        50.0    450.0      610.0      67.0    167.0          124.0
2020-12-20        44.0    346.0      610.0     130.0    235.0           53.0
2020-12-21        31.0    283.0      435.0     119.0    152.0           48.0
2020-12-22        63.0    484.0      852.0     208.0    314.0          141.0
...                ...      ...        ...       ...      ...            ...
2021-03-27       110.0    645.0     1398.0     168.0    389.0          235.0
2021-03-28       110.0    454.0     1559.0     408.0    795.0          155.0
2021-03-29       112.0    425.0        NaN     537.0   1091.0          129.0
2021-03-30       129.0      NaN        NaN     270.0    536.0          328.0
NaT                NaN      NaN        NaN     355.0    659.0          326.0

In [98]:


def hist_explore( pos ='latte1', bw=1 ):

  
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))

    ax.plot(data_test22[pos])
    
    plt.xlabel('Age')
    plt.ylabel('Density level')
    plt.title("Histogram for passengers age")
    plt.tight_layout()
    plt.show()


interact(hist_explore ,pos=['Celleneuve', 'Lattes2', 'Berracasa', 'Lavérune','Lattes1', 'Vieille_poste', 'Gerhardt', 'Tanneurs','Delmas1', 'Delmas2'] ,bw=(1, 6, 1))


interactive(children=(Dropdown(description='pos', options=('Celleneuve', 'Lattes2', 'Berracasa', 'Lavérune', '…

<function __main__.hist_explore(pos='latte1', bw=1)>